In [6]:
!pip install --upgrade git+https://github.com/huggingface/transformers
!pip install bitsandbytes
!pip install accelerate
!pip install auto-gptq
!pip install optimum
!pip install contractions

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 24.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.4/294.4 kB 31.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 10.5 MB/s eta 0:00:00
  Created wheel for optimum: filename=optimum-1.14.0-py3-none-any.whl size=398917 sha256=1b930346fcc1ce6e21eee4d722caf7735ef3d22ca32289d22d00dd3b0d245e58
  Stored in directory: /home/user/.cache/pip/wheels/b5/d3/b9/9667517bdcd7d15c43e804c8c0f88b006a85c7560f58e97ff5
Successfully built optimum


In [2]:
import torch
import transformers
from torch import cuda, bfloat16
import pandas as pd
import numpy as np
import contractions
import re
import json
import os
from json import JSONDecodeError

torch.cuda.set_device(0)

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'
device_name = torch.cuda.get_device_name()
print(f"Using device: {device} ({device_name})")

Using device: cuda:0 (NVIDIA RTX A6000)


In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch.fx
def initiateModelAndTokenizer():

    model_name_or_path = "TheBloke/Llama-2-7B-Chat-GPTQ"
    model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
                                                device_map = "auto",
                                                trust_remote_code = True,
                                                revision = "main")

    tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast = True)

    return model, tokenizer
model, tokenizer = initiateModelAndTokenizer()

In [4]:
def extractFeatures(review):

    user_prompt = f"""Extract most important product features from provided review. List all extracted features. Write the response as a Python list of dictionaries with three key-value pairs each. First key should be called "category" and value should be feature category. Second key should be called "quote" and value should be quote from review. Third key should be called "sentiment" and value should be "positive", "negative" or "neutral" depending on sentiment of the quoted text. Do not name the dictionary.
    Do not make up features if you will not find anything in review. The list may have one or more dictionaries. The list can be empty if there are no features mentioned in provided review. This is the review: {review}"""

    prompt = f"""[INST] <<SYS>> You are an objective assistant and your task is to help with extracting important information from product reviews. You only analyze information stated in provided text. You focus only on product features. If you cannot extract any product features then do not make up features which were not mentioned in provided text. <</SYS>>    {user_prompt}[/INST]
    """

    input_tokens = len(tokenizer.encode(prompt))

    input_ids = tokenizer(prompt, return_tensors = "pt").input_ids.cuda()
    start_time = time()
    output = model.generate(inputs = input_ids,
                            temperature = 0.01,
                            do_sample = True,
                            top_p = 0.01,
                            top_k = 20,
                            max_new_tokens = 1024)
    generation_time = round((time() - start_time), 2)

    output_tokens = len(output[0]) - input_tokens

    result = tokenizer.decode(output[0])

    result_list = convertResultToListOfDictionaries(result)
    
    # Dopisuje benchmark jako kolejny wiersz w pliku benchmarks.csv
    benchmark = [input_tokens, output_tokens, generation_time]
    with open(script_dir + "benchmarks/benchmarks.csv", "a") as f:
        writer_object = writer(f)
        writer_object.writerow(benchmark)

    return result_list



# Tworzenie tabeli z rankingiem produktów ze względu na liczbę recenzji rosnąco
def countReviewsPerProduct():

    products = reviews.groupby(["product_name"]).agg({"product_name": np.size})
    products.columns = ["number_of_reviews"]
    products = products.reset_index()
    products = products.sort_values(by = "number_of_reviews")
    products = products.reset_index(drop = True)

    return products



# Preprocessing tekstu każdej recenzji
def cleanText(text):

    # Zamienia powielone kropki na pojedynczą kropkę i spację
    text = re.sub("\.+", ". ", text)

    # Usuwa powielone spacje
    text = re.sub(" +", " ", text)

    # Usuwa white spaces z początku i końca
    text = text.strip()

    # Rozwija skróty, np. you're = you are
    text = contractions.fix(text)

    # Zamienia cudzysłów na apostrof
    text = text.replace("\"", "'")

    return text



# Konwersja odpowiedzi ze stringa do jsona
# Czasami odpowiedź jest źle sformatowana i wtedy wyskakuje błąd przy dekodowaniu,
# w takiej sytuacji do folderu errors jest zapisywana treść odpowiedzi czatu,
# żeby w przyszłości można było to przeanalizować i poprawić skuteczność konwersji
def convertResultToListOfDictionaries(result):

    # Wyciąga z odpowiedzi fragment z kodem i konwertuje go do listy słowników
    result = result.split("[/INST]")[1]
    result = result.split("[")[1]
    result = result.split("]")[0]
    result = "[" + result.strip() + "]"
    result = re.sub("},]", "}]", result)

    try:
        result_list = json.loads(result)
    except JSONDecodeError:
        with open(f"{script_dir}errors/{filename}.txt", "a") as f:
                    f.write(f"review_index: {r}\nJSONDecodeError:\n{result}\n\n\n")
        return result

    return result_list



# KONFIGURACJA

# Ścieżka do katalogu ze skryptem - punkt odniesienia do tworzenia pozostałych katalogów
script_dir = "/content/drive/MyDrive/Colab Notebooks/revbox_local/v2/"

# Ścieżka do pliku ze wszystkimi recenzjami
reviews_path = "/content/drive/MyDrive/Colab Notebooks/revbox_local/reviews/reviews.parquet"

# Liczba recenzji, po których zapisuje aktualne wyniki
# Chodzi o to, żeby nie stracić przetworzonych już danych w razie nieprzewidzianych błędów
save_checkpoint = 50

# Maksymalna liczba recenzji, które będą przetwarzane dla jednego produktu
reviews_cap = 10_000


# Inicjuje model i tokenizer
model, tokenizer = initiateModelAndTokenizer()

# Tworzy katalog z informacjami o błędach
if not os.path.exists(script_dir + "errors"):
    os.makedirs(script_dir + "errors")

# Tworzy katalog z wynikami
if not os.path.exists(script_dir + "results"):
    os.makedirs(script_dir + "results")

# Tworzy katalog z benchmarkami
if not os.path.exists(script_dir + "benchmarks"):
    os.makedirs(script_dir + "benchmarks")
    # Tworzy plik csv, do którego będą dopisywane benchmarki
    benchmarks_df = pd.DataFrame(columns = ["inputTokens",
                                            "outputTokens",
                                            "generationTime"])
    benchmarks_df.to_csv(script_dir + "benchmarks/benchmarks.csv", index = False)

# Wczytuje wszystkie recenzje z pliku
reviews = pd.read_parquet(reviews_path)

# Sprawdza czy istnieje plik z informacją o tym ile recenzji zostało już przetworzonych,
# jeśli nie ma, to tworzy go od nowa
if not os.path.exists(script_dir + "products.csv"):
    # Grupuje produkty według liczby recenzji
    products = countReviewsPerProduct()
    products["processed_reviews"] = 0
    products["next_index"] = 0
    products.to_csv(script_dir + "products.csv", index = False)
else:
    products = pd.read_csv(script_dir + "products.csv")

# Dla każdego produktu:
for p in range(len(products)):

    # Sprawdza czy wszystkie recenzje danego produktu zostały już przetworzone,
    # niektóre recenzje mogły być pominięte ze względu na błędy,
    # dlatego sprawdza jaki powinien być indeks następnej recenzji (początek od 0)
    # Sprawdza też czy nie został przekroczony limit maksymalnej liczby recenzji
    # dla jednego produktu
    if (products.loc[p, "next_index"] < products.loc[p, "number_of_reviews"]) and (products.loc[p, "processed_reviews"] < reviews_cap):

        product = products.loc[p, "product_name"]

        # Wydziela wszystkie recenzje wybranego produktu
        product_reviews = reviews.loc[reviews["product_name"] == product, ["review_title", "review_content"]].copy()
        product_reviews = product_reviews.reset_index(drop = True)

        # Czyści teksty recenzji
        product_reviews["review_content"] = product_reviews["review_content"].apply(cleanText)

        # Sprawdza czy istnieje już plik z wyekstraktowanymi cechami danego produktu
        filename = re.sub(" ", "_", product)
        if not os.path.exists(script_dir + "results/" + filename + ".csv"):
            # Tworzy dataframe, do którego będą dopisywane kategorie cech produktu oraz cytaty z recenzji
            features = pd.DataFrame()
        else:
            features = pd.read_csv(script_dir + "results/" + filename + ".csv")

        start_index = products.loc[p, "next_index"]
        processed_reviews = products.loc[p, "processed_reviews"]

        # Dla każdej recenzji:
        for r in range(start_index, len(product_reviews)):

            next_index = r + 1

            review = product_reviews.loc[r, "review_content"]

            # Stosuje algorytm do ekstrakcji cech i dopisuje do zbiorczej tabeli
            try:
                features_list = extractFeatures(review)
                # Jeśli zwróciło string zamiast listy, tzn. że był problem z konwersją
                # i przechodzi do kolejnej recenzji (informacje o błędzie są zapisywane)
                if type(features_list) == str:
                    continue
                features_tmp = pd.DataFrame(features_list)
                features = pd.concat([features, features_tmp])
            except Exception as e:
                error = type(e).__name__ + ":\n" + str(e)
                with open(f"{script_dir}errors/{filename}.txt", "a") as f:
                    f.write(f"review_index: {r}\n{error}\n\n\n")
                continue

            # Zapisuje aktualne wyniki jeśli osiągnęło ustalony checkpoint
            processed_reviews = processed_reviews + 1
            if processed_reviews % save_checkpoint == 0:
                features.to_csv(script_dir + "results/" + filename + ".csv", index = False)
                products.loc[p, "processed_reviews"] = processed_reviews
                products.loc[p, "next_index"] = next_index
                products.to_csv(script_dir + "products.csv", index = False)

            # Jeśli został osiągnięty limit liczby recenzji dla jednego produktu, to wychodzi z pętli recenzji
            # i przechodzi do zapisywania wyników, a następnie do kolejnego produktu
            if processed_reviews == reviews_cap:
                break

        features = features.reset_index(drop = True)
        features["category"] = features["category"].str.lower()
        features["sentiment"] = features["sentiment"].str.lower()

        # Zapisuje całość wyników dla danego produktu
        features.to_csv(script_dir + "results/" + filename + ".csv", index = False)
        products.loc[p, "processed_reviews"] = processed_reviews
        products.loc[p, "next_index"] = next_index
        products.to_csv(script_dir + "products.csv", index = False)